# 🚞 Zero-shot RE Training

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# # if you're running this in a colab notebook, you can run this cell to install the necessary dependencies
# !git clone https://github.com/jackboyla/GLiREL.git
# !cd GLiREL && pip install -e .
# !python -m spacy download en_core_web_sm

In [4]:
from glirel import GLiREL

save_path = 'logs/zero_rel/zero_rel-2024-06-07__20-37-25/model_42000'
model = GLiREL.from_pretrained(save_path)
# model

config.json not found in /home/jackboylan/GLiREL/logs/zero_rel/zero_rel-2024-06-07__20-37-25/model_42000
2024-06-08 05:54:32,662 - huggingface_hub.hub_mixin - WARNING - config.json not found in /home/jackboylan/GLiREL/logs/zero_rel/zero_rel-2024-06-07__20-37-25/model_42000


# Inference

To infer, the model needs `tokens`, `NER`, and `zero shot labels`.

### Eval data

In [5]:
import json
with open('./data/few_rel_all.jsonl', 'r') as f:
    data = [json.loads(line) for line in f]

i = 0

tokens = data[i]['tokenized_text']
ner = data[i]['ner']
labels = list(set([r['relation_text'] for r in data[i]['relations']]))
print(tokens)
print()
print(ner)
print(labels)

['Derren', 'Nesbitt', 'had', 'a', 'history', 'of', 'being', 'cast', 'in', '"', 'Doctor', 'Who', '"', ',', 'having', 'played', 'villainous', 'warlord', 'Tegana', 'in', 'the', '1964', 'First', 'Doctor', 'serial', '"', 'Marco', 'Polo', '"', '.']

[[26, 28, 'Q2989881', 'Marco Polo'], [22, 24, 'Q2989412', 'First Doctor']]
['characters']


In [6]:
labels = ['country of origin', 'licensed to broadcast to', 'father', 'followed by'] + labels
print(labels)

['country of origin', 'licensed to broadcast to', 'father', 'followed by', 'characters']


In [7]:
relations = model.predict_relations(tokens, labels, threshold=0.01, ner=ner)

print('Number of relations:', len(relations))  # num entity pairs (both directions) * num classes.... provided they're over the threshold

sorted_data_desc = sorted(relations, key=lambda x: x['score'], reverse=True)
print("\nDescending Order by Score:")
for item in sorted_data_desc:
    print(item)

Number of relations: 2

Descending Order by Score:
{'head_pos': [22, 24], 'tail_pos': [26, 28], 'head_text': ['First', 'Doctor', 'serial'], 'tail_text': ['Marco', 'Polo', '"'], 'label': 'followed by', 'score': 0.013861047104001045}
{'head_pos': [26, 28], 'tail_pos': [22, 24], 'head_text': ['Marco', 'Polo', '"'], 'tail_text': ['First', 'Doctor', 'serial'], 'label': 'followed by', 'score': 0.013023842126131058}


### Real-world example

In [8]:
# Real-world example
import spacy
nlp = spacy.load('en_core_web_sm')

text = "Apple Inc. was founded by Steve Jobs, Steve Wozniak, and Ronald Wayne in April 1976. The company is headquartered in Cupertino, California."

# text = "Jack Dorsey's father, Tim Dorsey, is a licensed pilot. Jack met his wife Sarah Paulson in New York in 2003. They have one son, Edward."

labels = [
    'co-founder', 
    'country of origin', 
    'licensed to broadcast to', 
    'no relation', 
    'parent', 
    'followed by', 
    'located in or next to body of water', 
    'spouse', 
    'child', 
    'founder', 
    'headquartered in', 
    'acquired by', 
    'subsidiary of'
    ]



def predict_and_show(text, labels):
    doc = nlp(text)
    print(f"Text: {text}")

    tokens = [token.text for token in doc]

    # NOTE: the end index should be inclusive
    ner = [[ent.start, (ent.end - 1), ent.label_, ent.text] for ent in doc.ents]
    print(f"Entities detected: {ner}")

    relations = model.predict_relations(tokens, labels, threshold=0.0, ner=ner, top_k=1)

    print('Number of relations:', len(relations))

    sorted_data_desc = sorted(relations, key=lambda x: x['score'], reverse=True)
    print("\nDescending Order by Score:")
    for item in sorted_data_desc:
        print(f"{item['head_text']} --> {item['label']} --> {item['tail_text']} | score: {item['score']}")

predict_and_show(text, labels)

Text: Apple Inc. was founded by Steve Jobs, Steve Wozniak, and Ronald Wayne in April 1976. The company is headquartered in Cupertino, California.
Entities detected: [[0, 1, 'ORG', 'Apple Inc.'], [5, 6, 'PERSON', 'Steve Jobs'], [8, 9, 'PERSON', 'Steve Wozniak'], [12, 13, 'PERSON', 'Ronald Wayne'], [15, 16, 'DATE', 'April 1976'], [23, 23, 'GPE', 'Cupertino'], [25, 25, 'GPE', 'California']]
Number of relations: 42

Descending Order by Score:
['Apple', 'Inc.'] --> headquartered in --> ['California'] | score: 0.9608165621757507
['Apple', 'Inc.'] --> headquartered in --> ['Cupertino'] | score: 0.9582273960113525
['California'] --> headquartered in --> ['Apple', 'Inc.'] | score: 0.9562973976135254
['Steve', 'Jobs'] --> headquartered in --> ['California'] | score: 0.9489175081253052
['Cupertino'] --> headquartered in --> ['Apple', 'Inc.'] | score: 0.9464631676673889
['Steve', 'Jobs'] --> headquartered in --> ['Cupertino'] | score: 0.9416676759719849
['Steve', 'Wozniak'] --> headquartered in --

In [9]:
text = "Jack knows Gill. They live in the same house in London. They are not related."
labels = ['family relation', 'knows', 'lives with', 'loves', 'licensed to broadcast to', 'father', 'followed by', 'no relation', 'lives in',]
predict_and_show(text, labels)

Text: Jack knows Gill. They live in the same house in London. They are not related.
Entities detected: [[0, 0, 'PERSON', 'Jack'], [2, 2, 'PERSON', 'Gill'], [11, 11, 'GPE', 'London']]
Number of relations: 6

Descending Order by Score:
['Gill'] --> lives in --> ['London'] | score: 0.9102070927619934
['Jack'] --> lives in --> ['London'] | score: 0.9041785001754761
['London'] --> lives in --> ['Gill'] | score: 0.8474307060241699
['London'] --> lives in --> ['Jack'] | score: 0.8287147879600525
['Jack'] --> knows --> ['Gill'] | score: 0.8061675429344177
['Gill'] --> knows --> ['Jack'] | score: 0.8060966730117798


In [10]:
# model.save_pretrained(
#     './release_model/glirel_beta', 
#     push_to_hub=True, 
#     repo_id='jackboyla/glirel_beta'
# )

In [11]:
import spacy
from spacy.tokens import Span
from spacy import displacy

def visualize_relation(text, relations):
    nlp = spacy.blank("en")
    doc = nlp(text)

    # Manually set dependency relations to visualize relations
    for token in doc:
        token.dep_ = "dep"  # default to 'dep'

    spans = []
    # Calculate character offsets for each entity
    for rel in relations:
        head = Span(doc, rel['head_pos'][0], rel['head_pos'][1], label=rel['head_text'])
        tail = Span(doc, rel['tail_pos'][0], rel['tail_pos'][1], label=rel['tail_text'])

        doc.ents += (head, tail)

        # Mock dependencies
        head_root = head.root
        tail_root = tail.root

        head_root.dep_ = "rel"  # Relation type can be customized
        head_root.head = tail_root  # Point head to tail

    options = {"fine_grained": True}
    displacy.render(doc, style="dep", options=options, jupyter=True)

# Example data
text = "Binsey located in or next to body of water River Thames"
relations = [
    {'head_pos': [0, 1], 'tail_pos': [9, 11], 'head_text': 'Binsey', 'tail_text': 'River Thames', 'label': 'located in or next to body of water', 'score': 0.9235768914222717},
    # {'head_pos': [9, 11], 'tail_pos': [0, 1], 'head_text': 'River Thames', 'tail_text': 'Binsey', 'label': 'located in or next to body of water', 'score': 0.12615662813186646}
]

visualize_relation(text, relations)
